In [1]:
from pyscf import gto,scf
from pyscf.geomopt.berny_solver import optimize
import numpy as np
from alch_deriv import alch_deriv
from FcMole import FcM
import matplotlib.pyplot as plt

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV
#ao_div=mol0.aoslice_by_atom()[:,-2:]
#pt=ao_div[1][0]
def change_order(P0,pt):  # to rotate the idx of the carbon atoms
    Pn=np.zeros_like(P0)
    Pn[:-pt,:-pt]=P0[pt:,pt:]
    Pn[-pt:,-pt:]=P0[:pt,:pt]
    Pn[:-pt,-pt:]=P0[pt:,:pt]
    Pn[-pt:,:-pt]=P0[:pt,pt:]
    return Pn

In [2]:
benz_atom_opt=""" C   0.000000   1.388317  0.000000;  
   H  -0.000000   2.461627  -0.000000  ;
   C   1.202318   0.694159   0.000000  ;
   H   2.131832   1.230814  -0.000000  ;
   C   1.202318  -0.694159  -0.000000  ;
   H   2.131832  -1.230814  -0.000000  ;
   C   0.000000  -1.388317   0.000000  ;
   H   0.000000  -2.461627   0.000000  ;
   C  -1.202318  -0.694159  -0.000000  ;
   H  -2.131832  -1.230814  -0.000000  ;
   C  -1.202318   0.694159   0.000000  ;
   H  -2.131832   1.230814  -0.000000  
"""
mol_benz = gto.M(atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
mf_benz=scf.RHF(mol_benz)
e_benz=mf_benz.scf()

converged SCF energy = -230.624474947017


In [3]:
P0=mf_benz.make_rdm1()
print(mol_benz.atom_charges())

[6 1 6 1 6 1 6 1 6 1 6 1]


In [20]:
d1,d2,d3=alch_deriv(mf_benz,dL=[[1,0],[-1,1]])
d1,d2,d3

(-14.008873027359318, -2.118464138927363, 1.2605035701457585)

In [21]:
fmol_benz = FcM(fcs=[[1,0],[-1,1]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz=scf.RHF(fmol_benz)
e_T=fmf_benz.scf()

converged SCF energy = -245.602645556279


In [22]:
e_benz+d1+1/2*d2+1/6*d3+fmol_benz.energy_nuc()-mol_benz.energy_nuc()

-245.5978403811996

In [23]:
#Finite Difference Coefficients
fdc1=np.asarray([1/12,-2/3,0,2/3,-1/12])
fdc2=np.asarray([-1/12,4/3,-5/2,4/3,-1/12])
fdc3=np.asarray([-1/2,1,0,-1,1/2])
fdc4=np.asarray([1,-4,6,-4,1])
coeffs=np.array([fdc1,fdc2,fdc3,fdc4])

In [24]:
dl=.1
Ps=[]
Es=[]
for i in range(-2,3):
    fm=FcM(fcs=[[1,0],[-1*dl*i,1*dl*i]],atom=benz_atom_opt, basis='6-31G')
    mf=scf.RHF(fm)
    Es.append(mf.scf(dm0=mf.init_guess_by_atom()))
    Ps.append(mf.make_rdm1())

converged SCF energy = -227.961951599782
converged SCF energy = -229.277086032536
converged SCF energy = -230.624474947017
converged SCF energy = -232.002903518768
converged SCF energy = -233.411089186009


In [25]:
dV=DeltaV(mol_benz,dL=[[1,0],[-1,1]])

In [26]:
e=e_benz+np.einsum('ij,ij',mf_benz.make_rdm1(),dV) +(fmf_benz.energy_nuc()-mf_benz.energy_nuc())
print("APDFT1",e,np.einsum('ij,ij',mf_benz.make_rdm1(),dV))
for i in range (4):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print("APDFT{}".format(i+2),e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

APDFT1 -244.74869224009353 -14.008873027359318
APDFT2 -245.80792647388927 -1.059234233795747
APDFT3 -245.5978475031208 0.21007897076847779
APDFT4 -245.56959662433795 0.02825087878282984
APDFT5 -245.60874582599914 -0.03914920166118579


In [27]:
print(e_T)
print(e_benz+d1+1/2*d2+1/6*d3+fmol_benz.energy_nuc()-mol_benz.energy_nuc() ,1/2*d2,1/6*d3)

-245.60264555627938
-245.5978403811996 -1.0592320694636814 0.2100839283576264


In [41]:
fmol_benz_1 = FcM(fcs=[[1,0],[-.01,.01]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz_1=scf.RHF(fmol_benz_1)
e_T_1=fmf_benz_1.scf()
d1_1,d2_1,d3_1=alch_deriv(fmf_benz_1,dL=[[1,0],[-1,1]])
fmol_benz_2 = FcM(fcs=[[1,0],[.01,-.01]],atom=benz_atom_opt, basis='6-31G') #angstroms not atomic units
fmf_benz_2=scf.RHF(fmol_benz_2)
e_T_2=fmf_benz_2.scf()
d1_2,d2_2,d3_2=alch_deriv(fmf_benz_2,dL=[[1,0],[-1,1]])

converged SCF energy = -230.76094180608
converged SCF energy = -230.488318540228


In [42]:
d4=(d3_1-d3_2)/.02
d5=(d3_1-2*d3+d3_2)/.01**2
print(d4/24,d5/120)

0.0282176920353099 -0.039837341927121774


In [43]:
print(e_T)
print(e_benz+d1+1/2*d2+1/6*d3+d4/24+d5/120+fmol_benz.energy_nuc()-mol_benz.energy_nuc() ,1/2*d2,1/6*d3)

-245.60264555627938
-245.60946003109143 -1.0592320694636814 0.2100839283576264
